In [1]:
from bs4 import BeautifulSoup
import datetime
import requests
import pickle

In [2]:
URL = "https://class-descriptions.northwestern.edu/4890/MEAS/COMP_SCI"
r = requests.get(URL)

In [3]:
soup=BeautifulSoup(r.content)
classes=soup.find('div',attrs={"class":"expander expander1"})


In [18]:
classesLinks=[x.find('a')["href"] for x in classes.find_all('li')]
classesTitles=[" ".join(x.text.strip().replace(" "," ").replace("\n","").split()) for x in classes.find_all('li')]

In [5]:
def getInfo(c):
    c="https://class-descriptions.northwestern.edu/4890/MEAS"+'/'+c
    c = requests.get(c)
    cSoup=BeautifulSoup(c.content)
    for x in cSoup.find_all('p'):
        if(":" in x.text and "-" in x.text):
            return x.text.replace("\n","").strip()
    return cSoup.find_all('p')[1].text.replace("\n","").strip()

In [6]:
informations=[getInfo(x) for x in classesLinks]

In [7]:
locations=[x[:x.index(":"):] for x in informations]

dayKey={"Mon":0,"Tues":1,"Wed":2,"Thurs":3,"Fri":4,"Sat":5,"Sun":6}

days=[x[x.index(":")+2:len(x)-x[::-1].index(",")-1] for x in informations]
days=[cDays.replace(' ','').split(',') for cDays in days]
days=[[dayKey[day] for day in cDays] for cDays in days]

timings=[x[len(x)-x[::-1].index(","):].replace(" ","").split('-') for x in informations]

In [96]:
def query(today=datetime.datetime.today().weekday(), totime=datetime.datetime.today().time()):
    results=[]
    startTimes=[]
    for ind, i in enumerate(days):
        frm=datetime.datetime.strptime(timings[ind][0],'%I:%M%p').time()
        to=datetime.datetime.strptime(timings[ind][0],'%I:%M%p').time()
        if(today in i and to>=totime):
            startTimes.append(frm)
            results.append([classesTitles[ind],locations[ind]])
    i=[]
    for i in zip( *sorted( zip(startTimes, results) ) ):
        pass
    sortArr = i
    return sortArr

In [76]:
query(2)

(['1: Game Design Studio - Robert Zubek - Mon, Wed, 6:00PM - 7:20PM',
  'Annenberg Hall G15'],)

In [88]:
with open('classesLinks.pkl', 'rb') as f: classesLinks=pickle.load(f)
with open('classesTitles.pkl', 'rb') as f: classesTitles=pickle.load(f)
with open('locations.pkl', 'rb') as f: locations=pickle.load(f)
with open('days.pkl', 'rb') as f: days=pickle.load(f)
with open('timings.pkl', 'rb') as f: timings=pickle.load(f)

In [98]:
query(0)

(['1: Game Design Studio - Robert Zubek - Mon, Wed, 6:00PM - 7:20PM',
  'Annenberg Hall G15'],
 ['9: Computer Science Education - Special Projects in Computer Science - Michael Stephen Horn - Paula Kay Hooper - Mon, 6:30PM - 9:20PM',
  'Annenberg Hall 303'])